This is an example of the process that I've been thinking we could use to do the crosswalk taking advantage of the site-level survey data.  It is just the bare bones and may be a little more complicated.  It assumes that all tables are neat and share column names, which they do not.  This example doesn't account for the confidence or compute a support metric.  That would come later.

First, build some example data frames for demonstration

In [7]:
import pandas as pd

# make example data frames - similar to what we already have
survey_df = pd.DataFrame([["a", "sp1", "forest"],
                         ["b", "sp2", "water"]],
                         columns=["site", "species", "WVtype"])

site_df = pd.DataFrame([["a", 1, 0], ["b", 0, 1]], 
                       columns=["site", "region1", "region2"])

cross_df = pd.DataFrame(columns=["WVtype", "GAPtype"], 
                        data=[["forest", "forest1"], ["forest", "forest2"],
                              ["forest", "forest3"], ["water", "water1"],
                              ["water", "water2"], ["water", "water3"]])

GAPlc_df = pd.DataFrame(columns=["GAPtype", "region1", "region2"], 
                        data=[["forest1", 1, 1], ["forest2", 1, 0],
                              ["forest3", 0, 1], ["water1", 1, 0],
                              ["water2", 1, 1], ["water3", 0, 1]])

print(survey_df); print("\n")
print(site_df); print("\n")
print(cross_df); print("\n")
print(GAPlc_df); print("\n")

  site species  WVtype
0    a     sp1  forest
1    b     sp2   water


  site  region1  region2
0    a        1        0
1    b        0        1


   WVtype  GAPtype
0  forest  forest1
1  forest  forest2
2  forest  forest3
3   water   water1
4   water   water2
5   water   water3


   GAPtype  region1  region2
0  forest1        1        1
1  forest2        1        0
2  forest3        0        1
3   water1        1        0
4   water2        1        1
5   water3        0        1




Add columns that represent crosswalk at wv level

In [8]:
df1 = pd.merge(GAPlc_df, cross_df, on=["GAPtype"], how="inner")
print(df1)

   GAPtype  region1  region2  WVtype
0  forest1        1        1  forest
1  forest2        1        0  forest
2  forest3        0        1  forest
3   water1        1        0   water
4   water2        1        1   water
5   water3        0        1   water


Assess a species.  I think it may be necessary to loop on regions (ecoregions or modeling regions) since they are in separate columns.  This example doesn't use a loop though, just the process that would be repeated in the looop.

In [9]:
sp = "sp1"
region = "region1"

# empty set for collection
linked_types = set([])

# pull out only the species data from the survey - this could be done earlier too
survey_df2 = survey_df[survey_df["species"] == sp]

# pull together site and survey data -- adds region columns
df2 = pd.merge(survey_df2, site_df, on=["site"], how="left") 
df2 = df2[df2[region] == 1]
print(df2)

  site species  WVtype  region1  region2
0    a     sp1  forest        1        0


Use the region column to pull out the slimmed down crosswalked list.  Note that if we hadn't used site location information, then there would have been 3 cover types resulting from the crosswalk.

In [10]:
df3 = pd.merge(df2, df1, on=["WVtype", region], how="left")
linked_types = linked_types | set(df3["GAPtype"].tolist())

# the final list
print(linked_types)

{'forest1', 'forest2'}
